# =========================================================
# 1_data_exploration.ipynb
# =========================================================

# Zelle 1: Bibliotheken importieren
import os
import pandas as pd
import matplotlib.pyplot as plt
import re

# Für Jupyter Notebook (inline Plots)
# (Falls du in Jupyter-Lab oder .ipynb bist)
# %matplotlib inline

# Optional: Falls du Tokenisierung testen möchtest
import nltk
nltk.download('punkt')

# ===========================================
# Zelle 2: Daten laden
# ===========================================
# Angenommen, du hast in data/raw/ verschiedene Märchentexte als TXT oder CSV.
# Wir zeigen hier ein Beispiel, wie du ggf. eine CSV mit "text" Spalte einlesen könntest.

RAW_DATA_DIR = "../data/raw"
sample_file = os.path.join(RAW_DATA_DIR, "fairytales_sample.csv")

df = pd.read_csv(sample_file)
print("Anzahl Datensätze:", len(df))
print("Erste Zeilen:")
display(df.head())

# ===========================================
# Zelle 3: Grober Überblick & Basis-Statistiken
# ===========================================
print("DataFrame Info:")
display(df.info())

print("Anzahl fehlender Werte pro Spalte:")
display(df.isnull().sum())

# Beispiel: Wir nehmen an, die "text"-Spalte enthält den Hauptinhalt der Märchen.
df["length_text"] = df["text"].apply(lambda x: len(str(x)))
print("\nStatistiken zur Textlänge (Zeichen):")
display(df["length_text"].describe())

# ===========================================
# Zelle 4: Erste Tokenisierung (Probe)
# ===========================================
# Wir können mal NLTK's word_tokenize testen,
# um zu sehen, wie unsere Texte in Tokens zerfallen.

def quick_tokenize(text):
    tokens = nltk.word_tokenize(str(text))
    return tokens

df["sample_tokens"] = df["text"].apply(lambda x: quick_tokenize(x[:2000])[:30])
# Hier beschränken wir uns auf die ersten 2000 Zeichen und zeigen 30 Tokens -> reiner Test

print("\nBeispiel-Tokenisierung (ersten 30 Tokens):")
display(df[["text", "sample_tokens"]].head())

# ===========================================
# Zelle 5: Häufigste Wörter & Visualisierung
# ===========================================
from collections import Counter

all_tokens = []
for text in df["text"].dropna():
    # Als einfache Demo: tokenize das gesamte Textfeld
    tokens = nltk.word_tokenize(str(text))
    all_tokens.extend(tokens)

word_freq = Counter(all_tokens)
most_common_30 = word_freq.most_common(30)
print("\nHäufigste 30 Wörter/Tokens:")
for token, freq in most_common_30:
    print(f"{token}: {freq}")

# Kleines Balkendiagramm
tokens_labels, tokens_counts = zip(*most_common_30)
plt.figure(figsize=(10,5))
plt.bar(tokens_labels, tokens_counts)
plt.xticks(rotation=45)
plt.title("Top 30 häufigste Tokens")
plt.show()

# ===========================================
# Zelle 6: Längenverteilung (Wörter)
# ===========================================
# Jetzt untersuchen wir, wie lang unsere Märchen typischerweise sind (in Wörtern).
def count_words(text):
    return len(nltk.word_tokenize(str(text)))

df["word_count"] = df["text"].apply(count_words)
print("\nWortanzahl-Statistiken:")
display(df["word_count"].describe())

# Histogramm zu Wortlängen
plt.figure(figsize=(8,4))
plt.hist(df["word_count"], bins=50)
plt.xlabel("Wortanzahl pro Märchen/Text")
plt.ylabel("Anzahl Datensätze")
plt.title("Verteilung der Wortanzahl")
plt.show()

# ===========================================
# Zelle 7: Beispiele für sehr lange/kurze Texte
# ===========================================
long_texts = df.nlargest(3, "word_count")
short_texts = df.nsmallest(3, "word_count")

print("\nBeispiele für sehr lange Märchen:")
display(long_texts[["text", "word_count"]])

print("\nBeispiele für sehr kurze Märchen:")
display(short_texts[["text", "word_count"]])

# ===========================================
# Zelle 8: Fazit & Ausblick
# ===========================================
# Basierend auf den Observations kannst du:
#  - Entscheiden, ob du bestimmte Ausreißer (zu kurz/zu lang) entfernen willst
#  - Ob du bestimmte Tokens oder Sonderzeichen bereinigen möchtest
#  - Wie groß dein Vokabular ungefähr wird
#  - Welche Sätze/Strukturen typisch sind, bevor du mit "2_data_preprocessing.ipynb" weitermachst.

print("\nFazit:\n- Daten exploriert, erste Statistiken über Textlängen und häufige Tokens erhalten.\n"
      "- Nächster Schritt: Daten säubern, Tokenisierung verfeinern, Splits erstellen (Notebook 2).")
